In [1]:
import numpy as np
import pandas as pd
import GPy
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance
import statsmodels.api as sm
import warnings
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import rankdata
from sklearn.cross_decomposition import CCA
import math
import copy
from math import log
from collections import Counter
import pickle
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
warnings.filterwarnings("ignore")

In [2]:
adjacency_matrix = np.load('adjacency_matrix.npy')

# Read beta back from the file
beta_star = np.load('beta_star.npy')
print("adjacency_matrix:", adjacency_matrix)
print("beta:", beta_star)

adjacency_matrix: [[0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0.]]
beta: [ 0.          0.          0.          0.          0.          0.
  0.          0.         11.92974738  0.          0.          0.
 -4.73132762  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.        

In [3]:
K = 2
error_std = 1

In [4]:
def transform_treatment(adjacency_matrix, A, K):
    # Extract upper triangular part (excluding diagonal)
    upper_triangular = np.triu(adjacency_matrix, k=1)
    
    # Generate B based on the rules
    B = []
    for i in range(len(adjacency_matrix)):
        for j in range(len(adjacency_matrix)):
            if upper_triangular[i, j] == 1:
                B.append(K * (A[i] - 1) + A[j])
    
    B = np.array(B)
    
    # Create X
    # Step 1: Start with 1
    X = [1]
    
    # Step 2: Add one-hot encoding of A (length K for each element in A)
    for a in A:
        one_hot_a = np.eye(K)[a - 1]  # One-hot encoding
        X.extend(one_hot_a)
    
    # Step 3: Add one-hot encoding of B (length K^2 for each element in B)
    for b in B:
        one_hot_b = np.eye(K**2)[b - 1]  # One-hot encoding
        X.extend(one_hot_b)
    
    X = np.array(X)

    return X

In [5]:
def linear_stochastic_bandit(lamada, T, K, adjacency_matrix, error_std, Y_t, X_t, beta_star, L, delta):
    
    n_features = X_t.shape[1]  # Number of features

    Y_t_exp = Y_t

    l_2_error_parameter = np.array([])

    for tau in range(len(Y_t), T):
        
        # Perform Ridge regression with lambda using Y_t_tran and X_t_tran
        ridge = Ridge(alpha=lamada, fit_intercept=False)
        ridge.fit(X_t, Y_t)  # Fit the model to get beta estimates
        beta_esti = ridge.coef_  # Estimated beta values
        #print(f"parameter square error:{np.sqrt(sum((beta_esti - beta_star)**2))}")

        # Perform optimization to find X_max
        # Check if expected Y_t has not changed for 20 iterations

        Gama = error_std * np.sqrt(2*(np.log(1/delta) + 0.5 * n_features * np.log(1 + tau * (L**2) / (n_features * lamada)))) + np.sqrt(lamada)*10
        # print(Gama)
        
        A = optimization(beta_esti, X_t, K, adjacency_matrix, Gama, tau)
        X_max = transform_treatment(adjacency_matrix, A, K)

        # Generate Y_max based on true beta_star
        Y_max = beta_star @ X_max + np.random.normal(0, error_std, 1)

        # Generate Y_max based on true beta_star
        Y_max_exp = beta_star @ X_max
    
        print(f"time {tau}: mean reward {beta_star @ X_max}")
        #print(f"time {tau}: action {X_max}")

        l_2_error_parameter = np.append(l_2_error_parameter, np.sqrt(sum((beta_esti - beta_star)**2)))

        # Append X_max and Y_max to X_t and Y_t
        Y_t = np.append(Y_t, Y_max)
        X_t = np.vstack([X_t, X_max])

        Y_t_exp = np.append(Y_t_exp, Y_max_exp)

    return Y_t, X_t, Y_t_exp, l_2_error_parameter


In [6]:
def optimization(beta_esti, X_t, K, adjacency_matrix, Gama, tau):
    # Define the objective function
    # Define the objective function
    def objective(X_max):
        X_max = np.array(X_max)
        sqrt_term = np.sqrt(X_max @ inv_gram @ X_max.reshape(-1, 1))
        return -(beta_esti @ X_max + Gama * sqrt_term)  # Negate for maximization
    
    # Initialize X_max with all zeros
    num_vars = len(adjacency_matrix)
    A = np.random.randint(1, K + 1, size=len(adjacency_matrix))
    
    X = transform_treatment(adjacency_matrix, A, K)

    inv_gram = np.linalg.pinv((1 / tau) * (X_t.T @ X_t))
    
    epochs = 5
    for epoch in range(epochs):
        current_value = objective(X)

        # Greedy optimization within this epoch
        for i in range(num_vars):
            # Flip the value of X_max[i] (0 -> L, L -> 0)
            original_value = A[i]

            
            # only useful for K = 2
            A[i] = 1 if original_value == 2 else 2
            
            X = transform_treatment(adjacency_matrix, A, K)
            new_value = objective(X)

            # If the objective improves, keep the change; otherwise, revert
            if new_value < current_value:
                current_value = new_value
            else:
                A[i] = original_value

    return A


In [7]:
def while_linear_stochastic_bandit(beta_star):
    error_std = 1
    delta = 0.01
    t = 2
    T = 1000 + t
    L = 1
    K = 2
    lamada = 1

    # Randomly generate two A arrays
    A1 = np.random.randint(1, K + 1, size=len(adjacency_matrix))
    A2 = np.random.randint(1, K + 1, size=len(adjacency_matrix))
    
    # Compute the treatment matrix for each A
    X_max1 = transform_treatment(adjacency_matrix, A1, K)
    X_max2 = transform_treatment(adjacency_matrix, A2, K)
    
    # Combine the results as the first two rows of X_t
    X_t = np.array([X_max1, X_max2])
    
    # Generate random error term
    error_t = np.random.normal(0, error_std, t)
    
    # Compute Y = X @ beta_star + error_t
    Y_t = X_t @ beta_star + error_t
    
    Y_T, X_T, Y_t_exp, l_2_error_parameter = linear_stochastic_bandit(lamada, T, K, adjacency_matrix, error_std, Y_t, X_t, beta_star, L, delta)

    return Y_t_exp, l_2_error_parameter

In [ ]:
Y_t_exp_times = []
l_2_error_parameter_times = []

# Run the experiment 10 times
for times in range(10):
    print(f"Running experiment {times + 1} for linear bandit")
    
    # Ensure `beta_star` and `t` are properly initialized before calling the function
    # Replace with actual initialization of beta_star and other parameters if needed
    Y_t_exp, l_2_error_parameter = while_linear_stochastic_bandit(beta_star)  # Replace with the actual function

    # Append the results of this experiment
    Y_t_exp_times.append(Y_t_exp)
    l_2_error_parameter_times.append(l_2_error_parameter)

# Convert lists to NumPy arrays for easier computation
Y_t_exp_times = np.array(Y_t_exp_times)
l_2_error_parameter_times = np.array(l_2_error_parameter_times)

# Save the results to .npy files
np.save(f'Y_t_exp_linear-stochastic-bandit-under-interference-n15-k2-d267-s10_10times.npy', Y_t_exp_times)
np.save(f'l_2_error_parameter_linear-stochastic-bandit-under-interference-n15-k2-d267-s10_10times.npy', l_2_error_parameter_times)
